#### connect to google palm

In [2]:
from langchain.llms import GooglePalm

api_key = ''

llm = GooglePalm(google_api_key=api_key, temperature=0.2)

/home/rafeed/Documents/Personal Education and Projects/ML Projects/Talking with database/Talking-with-a-database-LLM-project/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
poem = llm("write a poem about my love for roses")
print(poem)


* * *

Oh, roses, how I love thee!
Your beauty is beyond compare,
Your fragrance is like a dream,
Your petals soft and fair.

I love to watch you bloom,
Each day more beautiful than the last,
Your colors vibrant and alive,
Your form so graceful and elegant.

I love to smell your sweet scent,
It fills me with joy and peace,
It reminds me of all that is good in the world,
And it makes me feel like I'm home.

I love to touch your soft petals,
They feel like velvet against my skin,
They are so delicate and fragile,
Yet they are also so strong.

I love to admire your beauty,
You are a work of art,
You are a gift from nature,
And you are my one true love.

* * *


In [4]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [6]:
db_user = "root"
db_password = "password"
db_host = "localhost"
db_name = "rafeed_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [31]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("How many t-shirts do we have left for nike in extra small size and white color?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(35,)]
Answer:35
> Finished chain.


#### observations 1
The google palm model is smart enough to know that extra small size is the database column 'XS'. In both cases, nike and white color are small letters but in the database they are in capital letters

In [8]:
qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('227'),)]
Answer:227
> Finished chain.


#### observations 2
In the 2nd query, it gave wrong ans by applying sum to only price column. But in our db, price and stock_quantity columns need to be multiplied but the model thinks that the pricecs are for all of the quantity saved in price column. Here price is price per unit not the whole price for each category.

In [10]:
qns2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")




> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('14031'),)]
Answer:14031
> Finished chain.


In [11]:
qns3 = db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:SELECT SUM(price * (1 - pct_discount)) FROM discounts JOIN t_shirts ON discounts.t_shirt_id = t_shirts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() BETWEEN discounts.start_date AND discounts.end_date

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'discounts.start_date' in 'where clause'")
[SQL: SELECT SUM(price * (1 - pct_discount)) FROM discounts JOIN t_shirts ON discounts.t_shirt_id = t_shirts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() BETWEEN discounts.start_date AND discounts.end_date]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

#### observations 3
Here it generated wrong query and given an error as it assumes that we have discount starting and ending date available in the database columns. Usually, we see that a discount is available for a period of time, but here as in my database this column is not available, we will tell the llm to only focus on the columns we have in the database.

In [12]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('24796.100000'),)]
Answer:24796.1
> Finished chain.


It produced a correct answer when explicit query was given. 17462 is the total revenue without discounts. The total discount is 736.6. Hence revenue post discount is 17462-736.6=16725.4

Now this is not much interesting because what is the point of giving it the ready made query? Well, we will use this same query later on for few shot learning

In [13]:
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('27740'),)]
Answer:27740
> Finished chain.


In [14]:
qns5 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(90,), (31,), (13,)]
Answer:90
> Finished chain.


#### observations 4
Once again above is the wrong answer. We need to use SUM(stock_quantity). Let's run the query explicitly. We will use the result of this query later on in the notebook

In [15]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('134'),)]
Answer:134
> Finished chain.


### Few Shot Learning 

In [37]:
few_shots = [
 
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     },
]

### Creating Sementic Similarities based on Few Shot Learning

#### Procedures
1- Creating vector embeddings for each questions<br>
2- Storing embeddings in vecetor database chromadb<br>
3- Creating semantic similarity between questions

In [33]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [38]:
[example.values() for example in few_shots]

[dict_values(['How much is the total price of the inventory for all S-size t-shirts?', "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'", 'Result of the SQL query', '14031']),
 dict_values(['If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?', "SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n ", 'Result of the SQL query', '24796.1']),
 dict_values(['If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?', "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'", 'Result of the SQL query', '27740']),
 dict_values(["How many white color Levi's shirt I have?", "SELECT sum(stock_quantity) FROM t_sh

In [40]:
to_vectorize = [" ".join(example.values()) for example in few_shots]
to_vectorize

["How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query 14031",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query 24796.1",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi' Result of the SQL query 27740",
 "How many white color Levi's shirt I have? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White' Result of the SQL query 134"]

In [41]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)